In [5]:
%matplotlib inline

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import seaborn as sns
from scipy.interpolate import griddata
from plotting import plot_equatorial_pacific
from td_utils import clean_jma_data
from sst_utils import csv_from_aqua_modis_dataset, csv_from_avhrp_csv

from scipy.interpolate import griddata
from scipy.stats import gaussian_kde, pearsonr, spearmanr, ttest_ind
from scipy.spatial.distance import euclidean, cdist

from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import haversine_distances

The NCEI Marine data has no option of narrowing the target data field, i.e longitude and latitude of interest. Since it is of very large size, it was not used.

In [3]:
data = pd.read_csv('data/Marine_CSV_sample.csv')

In [4]:
data.columns

In [5]:
data[['Latitude', 'Longitude', 'Time of Observation', 'Sea Surface Temperature']]

Below is monthly data, collected from various stations in particular area, denoted by the coordinates located in the name of the file.

In [6]:
data = pd.read_csv('data/40_130_30_140.csv')

In [7]:
data.columns

In [8]:
data[['STATION', 'DATE', 'LATITUDE', 'LONGITUDE', 'SEA_SURF_TEMP']]

In [9]:
data.groupby(['DATE', 'LATITUDE', 'LONGITUDE'])['SEA_SURF_TEMP'].mean()

In [10]:
data['SEA_SURF_TEMP'][data['LATITUDE'] < 0]

In [11]:
data['LATITUDE'].max()

Below is monthly data downloaded from the https://neo.gsfc.nasa.gov/ website. It is monthly data in .nc format.

In [12]:
# reading the dataset for month of July 2024
# dataset = xr.open_dataset('data/aqua_modis.20240701_20240731.L3m.MO.SST.sst.9km.nc')

In [13]:
dataset = xr.open_dataset('data/aqua_modis/lanina/2010/AQUA_MODIS.20101201_20101231.L3m.MO.SST.sst.9km.nc')

In [14]:
# reading all variables
dataset.variables

Will have to make trasformation on the dataset in order to get the data needed

In [15]:
# extracting to pandas dataframe
raw_data = dataset['sst'].to_dataframe()

# droping the multiindex
tidy_data = raw_data.reset_index()

# filtering lat[20S; 20N]
tidy_data = tidy_data[(tidy_data.lat >= -20) & (tidy_data.lat <= 20)]

# filtering long[130E -> 180 -> -80 W]
tidy_data = tidy_data[(tidy_data.lon >= -180) & (tidy_data.lon <= -80) | (tidy_data.lon >= 130) & (tidy_data.lon <= 180)]

# droping na values
tidy_data = tidy_data.dropna()

# rounding lats and long to 0.5 deg
def round_to_nearest_half(x):
    return np.round(x * 2) / 2

tidy_data.lat = tidy_data.lat.apply(round_to_nearest_half)
tidy_data.lon = tidy_data.lon.apply(round_to_nearest_half)

# grouping by lat and long and averaging the sst
tidy_data = tidy_data.groupby(['lat', 'lon']).sst.mean()

# dropping multiindex
tidy_data = tidy_data.reset_index()

# getting the month
current_month = dataset.attrs['time_coverage_start']

# convert the string to pd.Timestamp object
current_month = pd.to_datetime(current_month)

# normalize timestamp
current_month = current_month.normalize()

# set the month as index of the cleaned data
tidy_data.index = pd.Index([current_month] * len(tidy_data))

In [16]:
# rounding the sst
# raw_data.sst.round(1)
# since the .round() is not working as expected, np used
# np.round(raw_data.sst, 1)
# since np is not working, using apply
tidy_data.sst = tidy_data.sst.apply(lambda x: round(x, 1))

In [17]:
tidy_data['lon'] = np.where(tidy_data['lon'] < 0, tidy_data['lon'] + 360, tidy_data['lon'])

In [18]:
tidy_data.sort_values(['lat', 'lon'])

In [19]:
# getting both the sst and the quality of the observation
combined_raw = dataset[['sst', 'qual_sst']].to_dataframe()

In [20]:
combined_raw = combined_raw.reset_index()

In [21]:
combined_raw.qual_sst.value_counts(dropna=False)

Not explicitly written which is the most reliable observation, however based on common conventions in quality flags, 0.0 is typically used to indicate the highest quality or most reliable observation. For the purpose of the project, the quality of the data is consider to be at sufficient level.

In [22]:
dataset

In [23]:
tidy_data

Lets try and plot the data to see how it looks:

In [24]:
plt.figure(figsize=(30, 10))

# contour map
lon_grid = np.linspace(tidy_data.lon.min(), tidy_data.lon.max(), 100)
lat_grid = np.linspace(tidy_data.lat.min(), tidy_data.lat.max(), 100)
lon_grid, lat_grid = np.meshgrid(lon_grid, lat_grid)
sst_grid = griddata((tidy_data.lon, tidy_data.lat), tidy_data.sst, (lon_grid, lat_grid), method='linear')
plt.contourf(lon_grid, lat_grid, sst_grid, cmap='jet', levels=20)


# plt.scatter(tidy_data.lon, tidy_data.lat, c=tidy_data.sst, cmap='jet', alpha=0.7, s=150, vmin=20, vmax=35)
plt.colorbar(label='Temperature')

x_tick=np.arange(130, 290, 10)
x_label=[f'{x}°E' if x <= 180 else f'{360 - x}°W' for x in x_tick]

y_tick = np.arange(-20, 25, 5)
y_label = [f'{np.abs(x)}°S' if x < 0 else f'{np.abs(x)}°N' for x in y_tick]

box = plt.Rectangle((190, -5), 50, 10, linewidth=2, edgecolor='RED', facecolor='none')
plt.gca().add_patch(box)

plt.axhline(y=0)

plt.xticks(ticks=x_tick, labels=x_label)
plt.yticks(ticks=y_tick, labels=y_label)

plt.xlim(130, 280)
plt.ylim(-20, 20)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title(f'Equatorian Distribution of SST for {current_month.month_name()}-{current_month.year}')  # Title of the plot
plt.show()  # Display the plot

The anomalies observed on the NE and SW quadrants of the diagram are false measurments taken at the earth surface.
We need to extract the python code of reading a dataset and converting it to clean and tidy csv to a function and in addition, to be able to read dirs reccursivly and apply the transformation to all datasets.

The below code is written in a separate module 'utils.py'

In [25]:
# result_list = extract_datasets('data/aqua_modis/elnino/2015/')

In [26]:
# dataset = result_list

In [27]:
# df = create_dataframe('esno_2015', *dataset)

In [28]:
df = pd.read_csv('data/csv_ready/elnino_2015.csv', index_col=0)

After having the combined dataframes I can extract each month data using a 'for' loop and anallyze the data:

In [29]:
for index, data in df.groupby(df.index):
    print(index, data.sst.mean())

In [30]:
for index, data in df.groupby(df.index):
    print(index, data.sst.median())

This is of no use. I need to filter the data for Niño 3.4 region (5°N-5°S, 170°W-120°W)

In [31]:
df_enso_region = df[(df.lat >= -5) & (df.lat <= 5) | (df.lon >= -170) & (df.lon <= -120)]

In [32]:
df_enso_region.index = pd.to_datetime(df_enso_region.index)

In [33]:
df_enso_region

In [34]:
for date, data in df_enso_region.groupby(df_enso_region.index):
    print(date, f'Mean: {data.sst.mean().round(2)}, Median: {data.sst.median()}')

Lets export plotting of the equatorial pacific ocean to a function

In [35]:
df_enso = df[(df.lat >= -5) & (df.lat <= 5) & (df.lon >= 190) & (df.lon <= 240)]

In [36]:
df.index = pd.to_datetime(df.index)

In [37]:
df.sst = df.sst.apply(lambda x: None if x > 33 else x)

In [38]:
df = df.dropna()

In [39]:
plot_equatorial_pacific(path='data/csv_ready/elnino_2015.csv', cond_name='El Nino')

Found out that my extracting function is not working as expected. Fixed it. I have the dataframe saved as expected.

Lets extract la nina and see how it works:

In [40]:
# lanina = extract_datasets('data/aqua_modis/lanina/2010/')

In [41]:
# lanina = create_dataframe('lanina_2010', *lanina)

In [42]:
lanina = csv_from_aqua_modis_dataset('data/aqua_modis/lanina/2010/', 'lanina_2010')

In [43]:
lanina.sst = lanina.sst.apply(lambda x: None if x > 33 else x)

In [44]:
lanina = lanina.dropna()

In [45]:
plot_equatorial_pacific('data/csv_ready/lanina_2010.csv', cond_name='La Nina')

The data before 2002 is provided from a different satelite. 
The data is in csv format, without indication of the lats and longs.

In [46]:
df = pd.read_csv('data/aqua_modis/elnino/1997/AVHRR_SST_M_1998-01-01_rgb_720x360.CSV', header=None)
lat = np.arange(-90, 90, 0.5)
lon = np.arange(-180, 180, 0.5)

In [47]:
df.columns = lon

In [48]:
df

In [49]:
df['lat'] = lat

In [50]:
df = df.melt(id_vars='lat', var_name='lon', value_name='sst')

In [51]:
df = df[(df.lat >= -20) & (df.lat <= 20)]

In [52]:
df = df[(df.lon >= -180) & (df.lon <= -80) | (df.lon >= 130) & (df.lon <= 180)]

In [53]:
df = df.reset_index(drop=True)

In [54]:
df.sst = df.sst.apply(lambda x: None if x > 100 else x)

In [55]:
df = df.dropna()

In [56]:
df['lon'] = np.where(df['lon'] < 0, df['lon'] + 360, df['lon'])

It is working as expected. Now to create a python function to read multiple files. Will modify the extract_database() function to read 

In [57]:
date = pd.to_datetime('1998-01')
df.index = pd.Index([date] * len(df))

In [58]:
# plot_equatorial_pacific(df, cond_name='El Nino')

In [59]:
result = csv_from_avhrp_csv('data/aqua_modis/elnino/1997/', 'elnino_1997', '1997-10')

In [60]:
result = result.dropna()

In [61]:
# plot_equatorial_pacific(result, 'El Nino')

Lets see how the temp changes over time for the ENSO region. First we get a function to apply ENSO region filter to the data:

In [62]:
tc_pacific = pd.read_csv('data/hurdat_pacific/hurdat2-nepac-1949-2023-042624.txt')

In [63]:
test = tc_pacific

In [64]:
# drop the index
test = test.reset_index()

In [65]:
# rename columns from 0 to len
test.columns = [i for i in range(len(test.columns))]

In [66]:
# drop columns 8 - end
test = test.drop(columns=[i for i in test.columns[8:]])

In [67]:
# name the columns
test.columns = ['date', 'time', 'consecutive_count', 'type_of_depression', 'lat', 'lon','max_wind_kn', 'min_pressure_mBar']

In [68]:
# changing time col to str 
test.time = test.time.astype(str)

In [69]:
# trimming time col values
test.time = test.time.apply(lambda x: x.strip())

In [70]:
test

In [71]:
def clean_ne_td_data(df):
    # reset index
    df = df.reset_index()
    # rename columns from 0 to len
    df.columns = [i for i in range(len(df.columns))]
    # drop columns 8 - end
    df = df.drop(columns=[i for i in df.columns[8:]])
    # name the columns
    df.columns = ['date', 'time', 'consecutive_count', 'type_of_depression', 'lat', 'lon', 'max_wind_kn',
                  'min_pressure_mBar']

    # modify the time
    df.time = df.time.astype(str)
    df.time = df.time.apply(lambda x: x.strip())

    # adding columns, removing non-observation entries
    df = export_name_basin_count(df)

    # continue to modify time
    df.time = df.time.apply(lambda x: x[0:2] + ':' + x[2:] + ':00')
    df.time = pd.to_timedelta(df.time)

    # strip the column, drop non-valid values and convert to int
    df.consecutive_count = df.consecutive_count.apply(lambda x: x.strip())
    df = df.drop(df.index[~df.consecutive_count.str.isdigit()])
    df = df.drop(df.index[~df.date.str.isdigit()])
    df.consecutive_count = df.consecutive_count.astype(int)

    # modify date, concatenate with time and set as index
    df.date = pd.to_datetime(df.date)
    df.date = df.date + df.time
    df.index = pd.Index(df.date)

    # drop the time and date
    df = df.drop(columns=['time', 'date'])
    
    # set the lat and lon
    df.lat = df.lat.apply(convert_lat_lon_to_number)
    df.lon = df.lon.apply(convert_lat_lon_to_number)

    # rearrange cols
    df = df[['basin', 'name', 'consecutive_count', 'type_of_depression', 'lat', 'lon', 'max_wind_kn',
       'min_pressure_mBar']]
    
    return df

def convert_lat_lon_to_number(value):
    return float(value[:-1]) if (value[-1] == 'N' or value[-1] == 'E') else float(value[:-1]) * -1 + 360

def export_name_basin_count(df):
    # adding additional cols for the type, name
    df['basin'] = pd.Series()
    df['name'] = pd.Series()
    
    # getting the rows with names
    names = df[~df.time.str.isdigit()]

    for index in names.index:
        basin = str(names.loc[index, 'date'])[:2]
        name = names.loc[index, 'time']
        count = names.loc[index, 'consecutive_count']

        start_index = index + 1
        while df.loc[start_index, 'time'].isdigit():
            df.loc[start_index, 'basin'] = basin
            df.loc[start_index, 'name'] = name
            df.loc[start_index, 'consecutive_count'] = count
            start_index += 1
            if start_index > df.index[-1]:
                break
        df = df.drop(index)
    return df

In [72]:
test = export_name_basin_count(test)

In [73]:
test

In [74]:
# strip the column and convert to int
test.consecutive_count = test.consecutive_count.apply(lambda x: x.strip())

In [75]:
# for this dropping non-valid rows
test = test.drop(test.index[~test.consecutive_count.str.isdigit()])
test = test.drop(test.index[~test.date.str.isdigit()])

In [76]:
# setting the type of the col
test.consecutive_count = test.consecutive_count.astype(int)

In [77]:
test.type_of_depression = test.type_of_depression.apply(lambda x: str(x).strip())

In [78]:
test.date = pd.to_datetime(test.date)

In [79]:
test.time = test.time.apply(lambda x: x[0:2] + ':' + x[2:] + ':00')

In [80]:
test.time = pd.to_timedelta(test.time)

In [81]:
test.date = test.date + test.time

In [82]:
test.index = pd.Index(test.date)

In [83]:
test = test.drop(columns=['time', 'date'])

In [84]:
def convert_lat_lon_to_number(value):
    return float(value[:-1]) if (value[-1] == 'N' or value[-1] == 'E') else float(value[:-1]) * -1 + 360

In [85]:
test.lat = test.lat.apply(convert_lat_lon_to_number)
test.lon = test.lon.apply(convert_lat_lon_to_number)

Now, it looks that the data is cleaned and tydied. We export all in function

In [86]:
test.min_pressure_mBar.min()

In [87]:
test2 = tc_pacific[-80:]

In [88]:
test_2 = pd.read_csv('data/hurdat_pacific/hurdat2-nepac-1949-2023-042624.txt')

In [89]:
# tt = clean_ne_td_data(test_2)

In [90]:
# tt.to_csv('data/csv_ready/ne_pacific_td.csv')

In [91]:
tt = pd.read_csv('data/csv_ready/ne_pacific_td.csv', index_col=0)

In [92]:
tt = tt.drop(tt.index[tt.min_pressure_mBar == -999])

In [93]:
tt.index = pd.to_datetime(tt.index)

In [94]:
tt[tt.index.year == 1959][-50:]

Time to clean the JMA data. It is not in very convenient format se we need to adjust for the columns

In [95]:
header_col_space = [5, 4, 3, 4, 4, 1, 20, 8]
extract_names = pd.read_fwf('data/jma_data/bst_all.txt', widths=header_col_space, header=None)

In [96]:
names = extract_names[extract_names[0] == 66666]

In [97]:
names

In [98]:
data_col_space = [8, 4, 2, 4, 5, 5, 13] # manually adjustd
full_data = pd.read_fwf('data/jma_data/bst_all.txt', widths=data_col_space, header=None, dtype='str')

In [99]:
full_data[-50:]

In [100]:
# def clean_jma_data(data_path='data/jma_data/bst_all.txt'):
#     # read the data and extract the header rows 
#     header_col_space = [5, 4, 3, 4, 4, 1, 20, 8]
#     extract_names = pd.read_fwf(data_path, widths=header_col_space, header=None)
#     names = extract_names[extract_names[0] == 66666]

#     # read the data and extract the header rows. the column space is manually adjusted
#     data_col_space = [8, 4, 2, 4, 5, 5, 13]
#     full_data = pd.read_fwf(data_path, widths=data_col_space, header=None, dtype='str')

#     # create a column to contain the names
#     full_data['name'] = pd.Series()

#     for index in names.index:
#         name = names.loc[index, 7]

#         start_index = index + 1
#         while True:
#             full_data.loc[start_index, 'name'] = name
#             start_index += 1
#             if start_index > full_data.index[-1]:
#                 break
#             if start_index in names.index:
#                 break
#         full_data = full_data.drop(index)

#     col_names = ['date', 'indicator', 'category', 'lat', 'lon', 'min_pressure_mBar', 'max_wind_kn', 'name']
#     full_data.columns = col_names
#     full_data = full_data.drop(columns='indicator')

#     full_data = cleaning_jma_columns(full_data)
#     full_data = modify_jma_date(full_data)

#     save_data(full_data, 'jma_td.csv')

#     return full_data

In [101]:
data = clean_jma_data()

In [104]:
data

In [105]:
# df = data

In [106]:
# df = df.drop(columns='indicator')

In [ ]:
# df.max_wind_kn = df.max_wind_kn.apply(lambda x: '000' if pd.isna(x) else x)

In [ ]:
# df.max_wind_kn = df.max_wind_kn.astype('int')

In [ ]:
# df.min_pressure_mBar = df.min_pressure_mBar.astype(int)

In [ ]:
# df.lat = df.lat.astype(int).apply(lambda x: x / 10)

In [ ]:
# df.lon = df.lon.astype(int).apply(lambda x: x / 10)

In [ ]:
# df.date = df.date.apply(jma_date_time)

In [ ]:
# df.index = pd.Index(df.date)

In [ ]:
# df.to_csv('data/csv_ready/jma_td.csv')

In [ ]:
# df = df.drop(columns='date')

In [ ]:
def save_data(data, filename):
    try:
        data.to_csv(f'data/csv_ready/{filename}.csv')
    except OSError:
        print('Error saving file')

def modify_jma_date(df):
    df.date = df.date.apply(jma_date_time)
    df.index = pd.Index(df.date)
    df = df.drop(columns='date')

    return df


def cleaning_jma_columns(df):
    # wind column
    df.max_wind_kn = df.max_wind_kn.apply(lambda x: '000' if pd.isna(x) else x)
    df.max_wind_kn = df.max_wind_kn.astype('int')

    # pressure column
    df.min_pressure_mBar = df.min_pressure_mBar.astype(int)

    # latitude
    df.lat = df.lat.astype(int).apply(lambda x: x / 10)

    # longitude
    df.lon = df.lon.astype(int).apply(lambda x: x / 10)

    return df


def jma_date_time(value):
    """ will be applied to all values in the column """

    date = value[:-2]
    time = value[-2:]

    time = pd.Timedelta(hours=int(time))

    date = f'{date[:2]}-{date[2:4]}-{date[-2:]}'

    if int(date[:2]) < 51:
        date = '20' + date
    else:
        date = '19' + date

    date = pd.to_datetime(date)

    date += time

    return date

In [ ]:
df_final = clean_jma_data()

In [ ]:
# df_final.to_csv('data/csv_ready/jma_td.csv')

In [ ]:
jma = pd.read_csv('data/csv_ready/jma_td.csv', index_col=0)

In [ ]:
jma.index = pd.to_datetime(jma.index)

Now that we have the complete data for the TD in north pacific, I will get geo data and convert it to csv so I can plot N Pacific

In [ ]:
jma[(jma.name == 'BOLAVE') & (jma.index.year == 2023)]

In [ ]:
jma[jma.index.year == 2023].groupby(['name']).value_counts()

In [ ]:
gdf = pd.read_csv('data/csv_ready/gdf_csv.csv')

In [ ]:
gdf = gdf.drop(columns=['id', 'source', 'level', 'parent_id', 'sibling_id', 'area', 'geometry'])

In [ ]:
gdf = gdf.drop(columns='geometry')

In [ ]:
gdf.lon = gdf.lon.apply(lambda x: x + 360 if x < 0 else x)

In [ ]:
gdf = gdf[gdf.lat <= 45]

In [ ]:
gdf = gdf[gdf.lat >= -20]

In [ ]:
gdf = gdf[gdf.lon >= 100]

In [ ]:
gdf = gdf[gdf.lon <= 290]

In [ ]:
plt.figure(figsize=(30, 10))
plt.scatter(gdf.lon, gdf.lat, s=0.5)

plt.show()

In [ ]:
# gdf.to_csv('data/csv_ready/gdf_pacific.csv', index=False)

In [ ]:
pd.read_csv('data/csv_ready/gdf_pacific.csv')

In [ ]:
jma_2023 = jma[jma.index.year == 2023]

In [ ]:
jma_2023[jma_2023.name == 'KHANU']

In [ ]:
plt.figure(figsize=(30, 10))
plt.scatter(gdf.lon, gdf.lat, s=0.5)

for td_name, values in jma_2023.groupby('name'):
    plt.plot(values['lon'], values['lat'], label=td_name)    

plt.show()

In [ ]:
from sst_utils import read_files, dataset_to_csv
from plotting import plot_equatorial_pacific

In [ ]:
result = read_files(directory_name='data/aqua_modis/neutral/', ext='nc')
# neutral_csv = dataset_to_csv('neutral_2012', result)

In [ ]:
neutral = pd.read_csv('data/csv_ready/neutral_2012.csv', index_col=0)

In [ ]:
neutral.index = pd.to_datetime(neutral.index)

In [ ]:
neutral.index = pd.to_datetime(neutral.index)neutral.index = pd.to_datetime(neutral.index)

In [ ]:
elnino = pd.read_csv('data/csv_ready/elnino_2015.csv', index_col=0)
elnino.index = pd.to_datetime(elnino.index)

In [ ]:
elnino

In [ ]:
# result = read_files(directory_name='data/aqua_modis/lanina/2010/', ext='nc')
# lanina = dataset_to_csv('lanina_2010', result)

In [ ]:
table = pd.read_table('data/year_period.txt', header=None)

In [ ]:
cols = ['date', 'djf','jfm','fma','mam','amj','mjj','jja','jas','aso','son','ond','ndj']
table.columns = cols

In [ ]:
table['enso'] = pd.Series()

In [ ]:
table.loc[0, 'enso'] = -1

In [ ]:
def apply_enso(df):
    for i in range(len(df)):
        if i == 0:
            continue
        if df.ndj[i - 1] > 0.5:
            df.loc[i, 'enso'] = 1
        elif df.ndj[i - 1] < -0.5:
            df.loc[i, 'enso'] = -1
        else:
            df.loc[i, 'enso'] = 0
    return df
table = apply_enso(table)

In [ ]:
table.enso.value_counts()

In [ ]:
cols = ['date', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 'enso']

In [ ]:
table.columns = cols

In [ ]:
table = table.melt(id_vars=['date', 'enso'], var_name='month', value_name='sst_anomaly').sort_values(['date', 'month'])

In [ ]:
def concat_year_month(df):
    df.date = df.date.astype(str)
    df.month = df.month.astype(str)
    for i in range(len(df)):
        month = df.loc[i, 'month']
        year = df.loc[i, 'date']
        if len(month) == 1:
            month = '0' + month
        df.loc[i, 'date'] = f'{year}-{month}'

    df.date = pd.to_datetime(df.date)
    df.index = pd.Index(df.date)
    df = df.drop(columns=['date','month'])
    return df
table = concat_year_month(table)

In [ ]:
# table.to_csv('data/csv_ready/oni_table.csv')

In [ ]:
table = pd.read_table('data/year_period.txt', header=None)

In [ ]:
cols = ['date', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
table.columns = cols

In [ ]:
table['sst_anomaly'] = pd.Series()

In [ ]:
def get_four_month_mean_anomaly(df):
    for i in range(len(df)):
        if i - 1 < 0:
            continue
        df.loc[i, 'sst_anomaly'] = (df.loc[i - 1, 10] + df.loc[i - 1, 11] + df.loc[i - 1, 12] + df.loc[i, 1]) / 4

    df = df.drop(columns=[i for i in range(1, 13)])
    return df

In [ ]:
table = get_four_month_mean_anomaly(table)

In [ ]:
# table.to_csv('data/csv_ready/oni_temp.csv', index=False)

In [ ]:
table

In [1]:
def get_dfs():
    oni_table = pd.read_csv('data/csv_ready/oni_table.csv', index_col=0)
    oni_temp = pd.read_csv('data/csv_ready/oni_temp.csv', index_col=0)
    oni_table.index = pd.to_datetime(oni_table.index)
    enso_phase = oni_table.groupby(oni_table.index.year)['enso'].apply(lambda x: x.unique()[0])

    jma = pd.read_csv('data/csv_ready/jma_td.csv', index_col=0)
    jma.index = pd.to_datetime(jma.index)
    frequency_jma = jma.groupby(jma.index.year)['name'].nunique()
    frequency_jma = pd.merge(frequency_jma, enso_phase, on='date')
    frequency_jma.columns = ['frequency', 'enso']

    nhc = pd.read_csv('data/csv_ready/ne_pacific_td.csv', index_col=0)
    nhc.index = pd.to_datetime(nhc.index)
    frequency_nhc = nhc.groupby(nhc.index.year)['name'].nunique()
    frequency_nhc = pd.merge(frequency_nhc, enso_phase, on='date')
    frequency_nhc.columns = ['frequency', 'enso']

    nhc_cp = nhc[nhc.basin == 'CP']
    frequency_nhc_cp = nhc_cp.groupby(nhc_cp.index.year)['name'].nunique()
    frequency_nhc_cp = pd.merge(frequency_nhc_cp, enso_phase, on='date')
    frequency_nhc_cp.columns = ['frequency', 'enso']

    nhc_ep = nhc[nhc.basin == 'EP']
    frequency_nhc_ep = nhc_ep.groupby(nhc_ep.index.year)['name'].nunique()
    frequency_nhc_ep = pd.merge(frequency_nhc_ep, enso_phase, on='date')
    frequency_nhc_ep.columns = ['frequency', 'enso']

    frequency_tables = [
        ('NW', frequency_jma),
        ('Central', frequency_nhc_cp),
        ('NE', frequency_nhc_ep)]

    dfs_freq = [frequency_jma, frequency_nhc]

    enso_phase_dt = enso_phase.copy()
    enso_phase_dt.index = pd.to_datetime(enso_phase.index.astype(str))

    merged = pd.merge(jma, enso_phase_dt, left_on=jma.index.year, right_on=enso_phase_dt.index.year, how='left')
    merged = merged.set_index(jma.index)
    jma_enso = merged.drop(columns='key_0')

    merged = pd.merge(nhc, enso_phase_dt, left_on=nhc.index.year, right_on=enso_phase_dt.index.year, how='left')
    merged = merged.set_index(nhc.index)
    nhc_enso = merged.drop(columns='key_0')

    gdf = pd.read_csv('data/csv_ready/gdf_pacific.csv')
    return jma_enso, nhc_enso

In [5]:
jma_enso, nhc_enso = get_dfs()

In [6]:
gridspace = 11

In [7]:
lat_grid = np.linspace(10, 20, gridspace)

In [8]:
lon_grid = np.linspace(0, 10, gridspace)

In [9]:
lon_grid, lat_grid = np.meshgrid(lon_grid, lat_grid)

In [10]:
for i in range(gridspace):
    for j in range(gridspace):
        plt.scatter(lon_grid[i, j], lat_grid[i, j])


In [22]:
lat = 18.5
lon = 2.5
gridspace = 11
lat_grid = np.linspace(10, 20, gridspace)
lon_grid = np.linspace(0, 10, gridspace)
plt.scatter(lon, lat)
plt.ylim(lat_grid.min(), lat_grid.max())
plt.yticks(lat_grid)
plt.xlim(lon_grid.min(), lon_grid.max())
plt.xticks(lon_grid)
plt.grid(visible=True, )
plt.show()